<a href="https://colab.research.google.com/github/ethvedbitdesjan/Fellowship/blob/master/2_Learning_Phrase_Representations_using_RNN_Encoder_Decoder_for_Statistical_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Please complete the missing parts in the code below. Moreover, please debug the code if the performance is not satisfactory.

## Objective - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this notebook, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). 

In [1]:
!pip install Torchtext==0.04      #installing the required version of Torchtext

     |████████████████████████████████| 61kB 4.9MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


## Preparing Data

We'll be coding up the models in PyTorch and using TorchText to help us do all of the pre-processing required. We'll also be using spaCy to assist in the tokenization of the data.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import torch.nn.functional as F

import spacy
import numpy as np

import random
import math
import time

We'll set the random seeds for deterministic results.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

First step is to create tokenizers. Eg - "Good Morning!" becomes ["Good", "morning", "!"].

spaCy has model for each language ("de" for German and "en" for English) which need to be loaded so we can access the tokenizer of each model. 

**Note**: the models must first be downloaded using the following on the command line: 
```
python -m spacy download en
python -m spacy download de
```

In [4]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
!python -m spacy download de

     |████████████████████████████████| 14.9MB 4.4MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=26c0344fdf072d1a0ec7b7ba0a8e192b6080fd1b82806aa3c7dcd008a3dd81eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-9emrgp99/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


Load both german and english models from spacy below.

In [6]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

Previously we reversed the source (German) sentence, however in the paper we are implementing they don't do this, so neither will we.

Next, we create the tokenizer functions. These can be passed to TorchText and will take in the sentence as a string and return the sentence as a list of tokens.

In [7]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [8]:
SRC = Field(tokenize=tokenize_de, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

Load our data.

In [9]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 686kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 169kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 164kB/s]


We can double check that we've loaded the right number of examples. They should be as follows:

Number of training examples: 29000

Number of validation examples: 1014

Number of testing examples: 1000

In [10]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [11]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

Finally, define the `device` and create our iterators.

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
BATCH_SIZE = 128

#Create an iterator below (using BucketIterator) to create your train, validation and test data.
train_iterator, valid_iterator, test_iterator = BucketIterator.splits( 
                                                (train_data, valid_data, test_data), 
                                                
                                                batch_sizes=(BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),

                                                sort_within_batch=True, 
                                                
                                                sort_key=lambda x: len(x.src), 
                                                
                                                device=device)

## Building the Seq2Seq Model

### Encoder

The encoder is similar to a multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last tutorial, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](assets/seq2seq5.png)

In [46]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):

        embedded = self.dropout(self.embedding(src))
        
        outputs, hidden = self.rnn(embedded)
        
        return hidden

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](assets/seq2seq6.png)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [47]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        input = input.unsqueeze(0)
                
        embedded = self.dropout(self.embedding(input))
                     
        emb_con = torch.cat((embedded, context), dim = 2)
                
        output, hidden = self.rnn(emb_con, hidden)
                
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)    
        
        prediction = self.fc_out(output)
    
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](assets/seq2seq7.png)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [48]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        if trg is None:
            trg = torch.zeros((25, src.shape[1])).fill_(2).long().to(src.device)
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [59]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 512
DEC_EMB_DIM = 512
HID_DIM = 512
ENC_DROPOUT = 0.3
DEC_DROPOUT = 0.3

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [60]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 512)
    (rnn): GRU(512, 512)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 512)
    (rnn): GRU(1024, 512)
    (fc_out): Linear(in_features=1536, out_features=5893, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [61]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,034,821 trainable parameters


We initiaize our optimizer.

In [62]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [63]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [64]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [65]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [66]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [67]:
def run(): # Made a function which can be used when improving the model
  N_EPOCHS = 10
  CLIP = 1

  best_valid_loss = float('inf')

  for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Finally, we test the model on the test set using these "best" parameters.

### The model should achieve a perplexity of 38 or lower on the test data.

In [68]:
### The model should achieve a perplexity of 58 or lower on the test data.model.load_state_dict(torch.load('tut2-model.pt'))
def testing():
  test_loss = evaluate(model, test_iterator, criterion)

  print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [69]:
run()

Epoch: 01 | Time: 0m 27s
	Train Loss: 4.933 | Train PPL: 138.845
	 Val. Loss: 4.986 |  Val. PPL: 146.319
Epoch: 02 | Time: 0m 28s
	Train Loss: 4.154 | Train PPL:  63.687
	 Val. Loss: 4.589 |  Val. PPL:  98.431
Epoch: 03 | Time: 0m 28s
	Train Loss: 3.666 | Train PPL:  39.111
	 Val. Loss: 4.181 |  Val. PPL:  65.436
Epoch: 04 | Time: 0m 28s
	Train Loss: 3.105 | Train PPL:  22.303
	 Val. Loss: 3.830 |  Val. PPL:  46.046
Epoch: 05 | Time: 0m 28s
	Train Loss: 2.665 | Train PPL:  14.364
	 Val. Loss: 3.777 |  Val. PPL:  43.701
Epoch: 06 | Time: 0m 28s
	Train Loss: 2.337 | Train PPL:  10.354
	 Val. Loss: 3.567 |  Val. PPL:  35.408
Epoch: 07 | Time: 0m 28s
	Train Loss: 2.066 | Train PPL:   7.890
	 Val. Loss: 3.488 |  Val. PPL:  32.711
Epoch: 08 | Time: 0m 28s
	Train Loss: 1.816 | Train PPL:   6.147
	 Val. Loss: 3.563 |  Val. PPL:  35.266
Epoch: 09 | Time: 0m 28s
	Train Loss: 1.629 | Train PPL:   5.096
	 Val. Loss: 3.548 |  Val. PPL:  34.739
Epoch: 10 | Time: 0m 28s
	Train Loss: 1.464 | Train PPL

In [70]:
testing()

| Test Loss: 3.595 | Test PPL:  36.401 |


### Please suggest any other ways/techniques/models that could improve the performance on this task.

A way to improve performance can be to use Attention in the model. The encoder in the previous model compressed the whole source into a single vector however using attention this does not need to be the case. A separate mechanism is used to identify the important part of the source. 

In [71]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):

        embedded = self.dropout(self.embedding(src))
        
        outputs, hidden = self.rnn(embedded)
        
        return outputs, hidden

In [72]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        # The input dimension is the concatenation of enc_hid_dim and dec_hid_dim
        self.attn_hidden_vector = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)

        # The attention scoring function
        self.attn_scoring_fn = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, enc_outputs):
        src_len = enc_outputs.shape[0]

        # We need to calculate the attn_hidden for each source words.
        # Repeat used instead of loop
        hidden = hidden.repeat(src_len, 1, 1)

        # Calculate Attention Hidden values
        attn_hidden = torch.tanh(self.attn_hidden_vector(torch.cat((hidden, enc_outputs), dim=2)))

        # Calculate the Scoring function. 
        attn_scoring_vector = self.attn_scoring_fn(attn_hidden).squeeze(2)

        # The attn_scoring_vector has dimension of [source len, batch size]
        # Since we need to calculate the softmax per record in the batch
        # we will switch the dimension to [batch size,source len]
        attn_scoring_vector = attn_scoring_vector.permute(1, 0)

        # Softmax function for normalizing the weights to
        # probability distribution
        return F.softmax(attn_scoring_vector, dim=1)



In [73]:
class Decoder(nn.Module):
    def __init__(self, output_dim,emb_dim, enc_hid_dim, dec_hid_dim, attention, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.hid_dim = enc_hid_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU(enc_hid_dim + emb_dim, dec_hid_dim)

        self.fc_out = nn.Linear(enc_hid_dim + dec_hid_dim + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, enc_outputs):
        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        # Calculating attention weights
        a = self.attention(hidden, enc_outputs).unsqueeze(1)

        # We need to shift the batch dimension in order to perform batch wise dot product.
        enc_outputs = enc_outputs.permute(1, 0, 2)

        # Calculating Weight W
        W = torch.bmm(a, enc_outputs)

        # Revert the batch dimension.
        W = W.permute(1, 0, 2)

        # concatenate the output with W
        emb_con = torch.cat((embedded, W), dim=2)

        output, hidden = self.rnn(emb_con, hidden)

        # Remove the sentence length dimension and pass them to the Linear layer
        preds = self.fc_out(torch.cat((output.squeeze(0), W.squeeze(0), embedded.squeeze(0)), dim=1))

        return preds, hidden, a.squeeze(1)


In [74]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        if trg is None:
            trg = torch.zeros((25, src.shape[1])).fill_(2).long().to(src.device)
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        enc_outputs, hidden = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden, a = self.decoder(input, hidden, enc_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [81]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 512
DEC_EMB_DIM = 512
HID_DIM = 512
ENC_DROPOUT = 0.3
DEC_DROPOUT = 0.3
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
attention = Attention(HID_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, HID_DIM, attention, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [82]:
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 512)
    (rnn): GRU(512, 512)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_hidden_vector): Linear(in_features=1024, out_features=512, bias=True)
      (attn_scoring_fn): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 512)
    (rnn): GRU(1024, 512)
    (fc_out): Linear(in_features=1536, out_features=5893, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [83]:
count_parameters(model)

20560133

In [84]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [85]:
run()

Epoch: 01 | Time: 0m 38s
	Train Loss: 4.926 | Train PPL: 137.845
	 Val. Loss: 4.922 |  Val. PPL: 137.340
Epoch: 02 | Time: 0m 39s
	Train Loss: 4.105 | Train PPL:  60.672
	 Val. Loss: 4.361 |  Val. PPL:  78.348
Epoch: 03 | Time: 0m 38s
	Train Loss: 3.370 | Train PPL:  29.084
	 Val. Loss: 3.722 |  Val. PPL:  41.367
Epoch: 04 | Time: 0m 38s
	Train Loss: 2.805 | Train PPL:  16.521
	 Val. Loss: 3.446 |  Val. PPL:  31.365
Epoch: 05 | Time: 0m 38s
	Train Loss: 2.399 | Train PPL:  11.017
	 Val. Loss: 3.322 |  Val. PPL:  27.725
Epoch: 06 | Time: 0m 38s
	Train Loss: 2.091 | Train PPL:   8.096
	 Val. Loss: 3.221 |  Val. PPL:  25.061
Epoch: 07 | Time: 0m 38s
	Train Loss: 1.832 | Train PPL:   6.247
	 Val. Loss: 3.238 |  Val. PPL:  25.488
Epoch: 08 | Time: 0m 38s
	Train Loss: 1.612 | Train PPL:   5.013
	 Val. Loss: 3.265 |  Val. PPL:  26.169
Epoch: 09 | Time: 0m 38s
	Train Loss: 1.498 | Train PPL:   4.472
	 Val. Loss: 3.274 |  Val. PPL:  26.406
Epoch: 10 | Time: 0m 38s
	Train Loss: 1.316 | Train PPL

In [86]:
testing()

| Test Loss: 3.395 | Test PPL:  29.809 |


As the test PPL shows, adding attention has improved the model.